In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import json

In [47]:
stockData = requests.get("https://query1.finance.yahoo.com/v8/finance/chart/MCD?region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=10y&corsDomain=finance.yahoo.com&.tsrc=finance")
stockJson = stockData.json()
stockResult = stockJson["chart"]["result"][0]
stockMetadata = stockResult["meta"]
stockPriceData = pd.DataFrame({"timestamp": stockResult["timestamp"]})
stockIndicators = pd.DataFrame(stockResult["indicators"]["quote"][0])
stockDataMerge = pd.concat([stockPriceData, stockIndicators], axis=1)
stockMetadata


{'currency': 'USD',
 'symbol': 'MCD',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': -110197800,
 'regularMarketTime': 1613163602,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 213.9,
 'chartPreviousClose': 76.14,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1613120400,
   'end': 1613140200,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1613140200,
   'end': 1613163600,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1613163600,
   'end': 1613178000,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '10y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [48]:
stockDataMerge

,timestamp,high,volume,close,low,open
0,1297693800,76.449997,4512800,76.239998,75.830002,76.070000
1,1297780200,76.339996,3823000,76.150002,75.800003,75.870003
2,1297866600,76.339996,4058200,76.019997,75.760002,76.279999
3,1297953000,76.129997,4338400,75.980003,75.699997,75.910004
4,1298039400,76.320000,5446900,76.129997,75.370003,76.080002
...,...,...,...,...,...,...
2512,1612794600,212.990005,2696200,211.580002,209.960007,212.529999
2513,1612881000,216.179993,3047300,215.979996,211.770004,212.070007
2514,1612967400,217.309998,3161600,214.399994,213.809998,216.880005
2515,1613053800,214.800003,2583300,214.270004,212.399994,214.539993


In [90]:
stockData = requests.get("https://finance.yahoo.com/quote/MCD/key-statistics?p=MCD")
soup = BeautifulSoup(stockData.content, 'html.parser')
job_elems = soup.find_all('section',  {"data-test":"qsp-statistics"})
for job_elem in job_elems:
    
        for t in job_elem.find_all("tbody"):
            print("**********************************")
            for td in t.find_all("td"):
                print(td.text)

**********************************
Market Cap (intraday) 5
159.38B
159.89B
163.54B
137.26B
122.95B
147.48B
Enterprise Value 3
204.45B
207.49B
212.21B
184.20B
169.60B
191.73B
Trailing P/E 
33.90
32.76
34.78
24.18
20.98
25.93
Forward P/E 1
25.19
25.32
27.03
32.15
19.53
23.42
PEG Ratio (5 yr expected) 1
3.06
3.08
3.68
4.82
2.58
2.83
Price/Sales (ttm)
8.35
8.47
8.66
6.75
6.00
7.29
Price/Book (mrq)
N/A
N/A
N/A
N/A
N/A
N/A
Enterprise Value/Revenue 3
10.64
39.05
39.17
48.97
35.98
35.84
Enterprise Value/EBITDA 6
26.69
93.50
71.60
132.66
79.03
70.40
**********************************
Beta (5Y Monthly) 
0.62
52-Week Change 3
-1.04%
S&P500 52-Week Change 3
16.75%
52 Week High 3
231.91
52 Week Low 3
124.23
50-Day Moving Average 3
211.82
200-Day Moving Average 3
214.14
**********************************
Avg Vol (3 month) 3
3.19M
Avg Vol (10 day) 3
2.83M
Shares Outstanding 5
745.11M
Float 
750.38M
% Held by Insiders 1
0.06%
% Held by Institutions 1
68.80%
Shares Short (Jan 28, 2021) 4
5.73M
Short Ra